In [1]:
## IT'S DANGEROUS TO GO ALONE! TAKE THIS:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
pd.set_option('display.max_columns', None)

### Housekeeping notes:

All Notebooks should be available on github at (you may be here already):
- https://github.com/gweggurs/Soil


All data required to run the notebooks is available as a zipped folder via Google Drive:
- https://drive.google.com/drive/folders/1tSi-8RoeVKNt2BNSTHtNHD27xPzUxOr7?usp=sharing

*tl;dr:
Check the sixth notebook for conclusions after modeling and the seventh notebook for the answer to the initial question of "Who can benefit the most from converting to regenerative agriculture?"*

---

# Let the Soil Play its Simple Part
### Using Analysis and Predictive Modeling to find who could benefit the most from converting to Regenerative Agriculture practices.
---

This project started with the question: 
    
   **Who can benefit the most from converting to regenerative agriculture?**
   
Regenerative agriculture means different things depending on context, but the principles I decided to focus on were:

1. No-till farming to prevent topsoil erosion and to leave the soil biome undisturbed
2. Swapping manufactured Chemical and Fertilizer inputs for increased biodiversity and livestock fertilization
3. Cover cropping to enhance water infiltration and further prevent erosion 

No-Till agricultural practices and cover cropping have both been shown to have positive effects on soil health, reducing erosion, and improved moisture regulation--helping with both extreme drought and extreme moisture conditions.

As I dug into the question I decided to keep the scope limited to the United States. I wanted to have as much granularity as possible in defining the 'Who' in the question and decided to try to get data down to the county level for as many states in the USA as I could. 

I initially found data for the USDA's Census of Agriculture. The Census runs every 5 years and returns a myriad of agricultural statistics for all 50 states and the roughly 3,000 counties within the states. I found data for the total cost of Chemical and fertilizer inputs and decided to use this as my operating definition for the "amount" of conventional agriculture in a given county. These variables also provided the first data point for triaging the county or state that could benefit by  converting from conventional agriculture to regenerative:

   * **What areas have the highest costs associated with conventional Agriculture?**

With the Census data, I would be able to determine what states or counties spend the most on Chemicals and fertilizers, and therefore, the place that would be able to reduce their overhead costs the most if they removed those input from their way of farming.

The other aspect I wanted to address was risk. I found some data on changing weather patterns and debated using soil health measure or ground water levels, but in the end each area I examined for agricultural risk factors pointed toward drought. I found the US Drought Monitor and decided to use this as my second way of locating an area that would benefit from regenerative agriculture:
    
   * **What areas have the highest risk of drought?**

With the Drought Monitor data I would be able to find the counties or states with the most frequent droughts--in other words, the places that could benefit the most from *mitigating* drought with improved soil health and water infiltration.

The final piece I wanted to incorporate into my project was predictive modeling of drought with machine learning. The modeling would include location data,  agricultural data, and the addition of basic weather data to predict the target variable of drought. Attempting to predict drought would potentially yield two benefits. 

First, as long as I would be able to check what factors were most important in predicting drought, I could use those factors to further triangulate my search for the area in greatest need of cost and risk reduction. 

Second, if the features used to predict drought allow for enough time for proaction, then the model could be used to catch warning signs of drought and give forewarning for at-risk areas.

The modeling would provide the final piece:

   * **What factors are most important for predicting drought?**
   
In the end, I succeeded in answering each of these questions with insights from analysis and interpretation of machine learning models.


# Soil, Data Acquisition

**Let the Soil Play its Simple Part**

Greg Sakowski
* Book 1 of 7
* Discussing downloaded weather data and formatting it to csv. Scraping a small table from Wikipedia. Pulling Drought data from US Drought Monitor API. Discussing USDA spreadsheet downloaded via self-serve database. 
* Reading from CSV: climdiv-tmincy-v1.0.0-20220907.txt, climdiv-tmaxcy-v1.0.0-20220907.txt, climdiv-tmpccy-v1.0.0-20220907.txt, climdiv-pcpncy-v1.0.0-20220907.txt, FIPS_to_ClimDiv.csv, 
* Writing to CSV: roughMinTemp.csv, roughMaxTemp.csv, roughAvgTemp.csv, roughPrecip.csv, roughFIPSandClimDiv_df.csv, roughDroughtFIPS.csv
* **Warning: The Drought section has two api scrapes that will run for 2 - 4 hours total. There is a shorter 'test scrape' that can work as proof-of-concept.

---
## Table of Contents:

[Monthly Weather by County](#Monthly-Weather-by-County)

[County Data Table via Wikipedia](#County-Data-Table-via-Wikipedia)

[US Drought Monitor](#US-Drought-Monitor)

[USDA Census of Agriculture Data](#USDA-Census-of-Agriculture-Data)


# Monthly Weather by County 
---

The data documentation from NCEI for this county-level data can be found at:

https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-readme.txt

The data itself can be accessed at:

https://www.ncei.noaa.gov/pub/data/cirs/climdiv/

We have 4 types of weather data:

**Minimum temperature**: 
    
with the first column containing the 5 digit FIPS id, followed by the number 28 to indicate the data was the Minimum temperature for that month, and the 4 digit year at the end: 
    **27053282014** would be Hennepin county's monthly minimums for 2014

**Maximum Temperature**: 
    
with the first column containing the 5 digit FIPS id, followed by the number 27 to indicate the data was the Maximum temperature for that month, and the 4 digit year at the end: 
    **27053272014** would be Hennepin county's monthly maximums for 2014
    
**Average Temperature**: 
    
with the first column containing the 5 digit FIPS id, followed by the number 02 to indicate the data was the Minimum temperature for that month, and the 4 digit year at the end: 
    **27053022014** would be Hennepin county's monthly averages for 2014
    
**Precipitation**:

with the first column containing the 5 digit FIPS id, followed by the number 01 to indicate the data was the total precipitation for that month, and the 4 digit year at the end: 
    **27053012014** would be Hennepin county's total monthly Precipitation for 2014
    
    
We can get monthly data at the county level for each of measures. Each measure was accessed on the ncei.noaa site and was saved as a .txt file. We will add a header when we import the data:

    **FIPSxxYear 01 02 03 04 05 06 07 08 09 10 11 12**

   wherein the xx divider between the FIPS number and year corresponded with the measurement. 

Because our FIPS numbers contain leading zeros, we can force the first column to be an object to keep our leading zeros in place.

There are a few other measures available that can be incorporated later on if needed:
 - Heating Degree Days
 - Cooling Degree Days
 - 'Normals' for all six measures

The Cooling days and heating days aren't likely to be necessary for our analysis. They refer to deviation from a 65 degree average, with the assumption that under 65 degrees a person would need to heat their home and above 65 they would need to cool it.

The documentation is clear that the 'Normals' for this dataset are not true normals as defined by NCEI, but are 30 year averages, 100 year averages, etc. We may use some of these unofficial normals to compare our data to for reference, but for now we can move ahead with loading in the data.

In [2]:
#author note--the original weather files were downloaded in mid-august and had a timestamp of 20220804,
#the author (foolishly) deleted the original files once they were in csv format, 
#hence a later-than-expected timestamp of 20220907. 
minTemp_df = pd.read_csv('data/climdiv-tmincy-v1.0.0-20220907.txt',
                          sep='\s+',
                          names = ['FIPS28Year', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
                          index_col=False, 
                          dtype={'FIPS28Year':str})

minTemp_df

,FIPS28Year,1,2,3,4,5,6,7,8,9,10,11,12
0,01001281895,34.2,27.7,43.4,51.8,59.3,67.4,69.7,70.3,67.1,46.9,42.1,32.5
1,01001281896,34.4,37.2,42.6,57.0,65.0,67.9,71.4,71.7,65.0,52.2,46.1,35.9
2,01001281897,33.2,41.5,51.2,50.9,56.8,69.2,71.4,69.3,64.4,53.4,41.7,37.7
3,01001281898,39.6,34.4,49.1,47.1,60.4,69.1,70.2,69.6,65.7,50.6,38.6,32.7
4,01001281899,33.6,29.6,44.3,51.3,64.1,68.4,69.9,70.4,61.1,54.8,43.2,34.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400631,50290282018,-9.1,-1.1,6.1,16.1,34.4,44.4,49.9,44.0,35.6,23.6,6.8,-3.7
400632,50290282019,-9.0,1.2,16.8,19.8,36.8,48.0,51.9,42.8,36.0,21.6,5.1,-11.4
400633,50290282020,-22.8,-16.7,-1.6,18.4,36.3,45.9,47.1,44.8,34.9,18.3,-0.9,-3.2
400634,50290282021,-3.2,-17.2,-6.3,11.0,33.4,46.3,49.2,43.9,31.0,20.4,-7.4,-7.8


In [3]:
minTemp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400636 entries, 0 to 400635
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FIPS28Year  400636 non-null  object 
 1   1           400636 non-null  float64
 2   2           400636 non-null  float64
 3   3           400636 non-null  float64
 4   4           400636 non-null  float64
 5   5           400636 non-null  float64
 6   6           400636 non-null  float64
 7   7           400636 non-null  float64
 8   8           400636 non-null  float64
 9   9           400636 non-null  float64
 10  10          400636 non-null  float64
 11  11          400636 non-null  float64
 12  12          400636 non-null  float64
dtypes: float64(12), object(1)
memory usage: 39.7+ MB


In [4]:
maxTemp_df = pd.read_csv('data/climdiv-tmaxcy-v1.0.0-20220907.txt', 
                         sep='\s+', 
                         index_col=False,
                         names = ['FIPS27Year', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
                         dtype={'FIPS27Year':str})
maxTemp_df

,FIPS27Year,1,2,3,4,5,6,7,8,9,10,11,12
0,01001271895,53.7,48.7,67.6,76.4,81.9,89.2,91.1,90.4,90.9,76.0,66.6,58.0
1,01001271896,54.2,60.8,65.3,81.6,88.5,88.2,92.0,94.5,90.8,77.2,69.9,58.7
2,01001271897,54.2,63.1,71.4,75.1,83.2,95.6,93.3,89.9,88.9,81.3,68.1,58.8
3,01001271898,60.6,59.1,71.0,72.0,89.5,93.9,91.5,88.8,86.7,73.6,61.7,55.7
4,01001271899,55.6,53.4,68.8,73.4,89.3,93.7,92.2,92.6,87.5,78.4,68.1,56.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400631,50290272018,4.6,13.4,24.2,35.2,52.9,64.8,70.2,57.9,51.7,37.2,16.6,9.1
400632,50290272019,5.4,17.6,33.2,38.7,57.3,70.7,71.9,60.6,51.8,33.4,16.4,1.6
400633,50290272020,-9.6,2.0,17.3,36.2,59.1,66.0,64.8,65.3,49.4,31.4,12.7,9.4
400634,50290272021,10.4,-0.7,15.1,34.9,55.2,66.4,66.8,58.2,47.2,31.0,4.2,7.6


In [5]:
maxTemp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400636 entries, 0 to 400635
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FIPS27Year  400636 non-null  object 
 1   1           400636 non-null  float64
 2   2           400636 non-null  float64
 3   3           400636 non-null  float64
 4   4           400636 non-null  float64
 5   5           400636 non-null  float64
 6   6           400636 non-null  float64
 7   7           400636 non-null  float64
 8   8           400636 non-null  float64
 9   9           400636 non-null  float64
 10  10          400636 non-null  float64
 11  11          400636 non-null  float64
 12  12          400636 non-null  float64
dtypes: float64(12), object(1)
memory usage: 39.7+ MB


In [6]:
avgTemp_df = pd.read_csv('data/climdiv-tmpccy-v1.0.0-20220907.txt', 
                         sep='\s+', 
                         index_col=False,
                         names = ['FIPS02Year', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
                         dtype={'FIPS02Year':str})
avgTemp_df

,FIPS02Year,1,2,3,4,5,6,7,8,9,10,11,12
0,01001021895,44.0,38.2,55.5,64.1,70.6,78.3,80.4,80.4,79.0,61.4,54.4,45.3
1,01001021896,44.3,49.0,54.0,69.3,76.8,78.0,81.7,83.1,77.9,64.7,58.0,47.3
2,01001021897,43.7,52.3,61.3,63.0,70.0,82.4,82.4,79.6,76.6,67.4,54.9,48.2
3,01001021898,50.1,46.8,60.1,59.6,75.0,81.5,80.8,79.2,76.2,62.1,50.2,44.2
4,01001021899,44.6,41.5,56.6,62.3,76.7,81.0,81.0,81.5,74.3,66.6,55.7,45.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400631,50290022018,-2.2,6.1,15.1,25.6,43.6,54.6,60.0,51.0,43.6,30.4,11.7,2.7
400632,50290022019,-1.8,9.4,25.0,29.2,47.0,59.3,62.0,51.7,43.9,27.5,10.7,-4.9
400633,50290022020,-16.2,-7.4,7.9,27.3,47.7,55.9,55.9,55.1,42.1,24.8,5.9,3.1
400634,50290022021,3.6,-9.0,4.4,22.9,44.3,56.4,58.0,51.1,39.2,25.7,-1.6,-0.1


In [7]:
avgTemp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400636 entries, 0 to 400635
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FIPS02Year  400636 non-null  object 
 1   1           400636 non-null  float64
 2   2           400636 non-null  float64
 3   3           400636 non-null  float64
 4   4           400636 non-null  float64
 5   5           400636 non-null  float64
 6   6           400636 non-null  float64
 7   7           400636 non-null  float64
 8   8           400636 non-null  float64
 9   9           400636 non-null  float64
 10  10          400636 non-null  float64
 11  11          400636 non-null  float64
 12  12          400636 non-null  float64
dtypes: float64(12), object(1)
memory usage: 39.7+ MB


In [8]:
precip_df = pd.read_csv('data/climdiv-pcpncy-v1.0.0-20220907.txt', 
                         sep='\s+', 
                         index_col=False,
                         names = ['FIPS01Year', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'],
                         dtype={'FIPS01Year':str})
precip_df

,FIPS01Year,1,2,3,4,5,6,7,8,9,10,11,12
0,01001011895,7.03,2.96,8.36,3.53,3.96,5.40,3.92,3.36,0.73,2.03,1.44,3.66
1,01001011896,5.86,5.42,5.54,3.98,3.77,6.24,4.38,2.57,0.82,1.66,2.89,1.94
2,01001011897,3.27,6.63,10.94,4.35,0.81,1.57,3.96,5.02,0.87,0.75,1.84,4.38
3,01001011898,2.33,2.07,2.60,4.56,0.54,3.13,5.80,6.02,1.51,3.21,6.66,3.91
4,01001011899,5.80,6.94,3.35,2.22,2.93,2.31,6.80,2.90,0.63,3.02,1.98,5.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
400631,50290012018,0.47,1.04,0.80,0.80,1.36,1.97,1.71,4.58,1.95,1.43,0.95,0.88
400632,50290012019,0.52,1.31,0.93,0.52,1.18,1.08,1.67,3.56,2.54,2.27,2.25,0.58
400633,50290012020,0.34,0.71,1.04,1.58,0.39,2.45,2.46,2.06,2.57,0.94,1.12,0.52
400634,50290012021,0.43,0.63,0.86,0.95,0.53,1.82,2.82,4.48,1.96,1.40,0.90,3.19


In [9]:
precip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400636 entries, 0 to 400635
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   FIPS01Year  400636 non-null  object 
 1   1           400636 non-null  float64
 2   2           400636 non-null  float64
 3   3           400636 non-null  float64
 4   4           400636 non-null  float64
 5   5           400636 non-null  float64
 6   6           400636 non-null  float64
 7   7           400636 non-null  float64
 8   8           400636 non-null  float64
 9   9           400636 non-null  float64
 10  10          400636 non-null  float64
 11  11          400636 non-null  float64
 12  12          400636 non-null  float64
dtypes: float64(12), object(1)
memory usage: 39.7+ MB


In [10]:
# sending all of the weather dfs to csv's
minTemp_df.to_csv('roughMinTemp.csv', index=False)
maxTemp_df.to_csv('roughMaxTemp.csv', index=False)
avgTemp_df.to_csv('roughAvgTemp.csv', index=False)
precip_df.to_csv('roughPrecip.csv', index=False)

# County Data Table via Wikipedia:
A whole lot of kudos go out to a wiki user that compiled a table of helpful information for each US county.
https://en.wikipedia.org/wiki/User:Michael_J/County_table

In [11]:
html_doc = requests.get('https://en.wikipedia.org/wiki/User:Michael_J/County_table').content

In [12]:
countyTable = pd.read_html(html_doc)
countyTable[0]

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,1001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,+32.536382°,–86.644490°
1,2,AL,1003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.190,437.527,5250.712,2027.311,+30.659218°,–87.746067°
2,3,AL,1005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,+31.870670°,–85.405456°
3,4,AL,1007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,+33.015893°,–87.127148°
4,5,AL,1009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,+33.977448°,–86.567246°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3139,WY,56037,Sweetwater,Green River,43806,27004.897,10426.649,166.887,64.436,27171.784,10491.085,+41.660339°,–108.875676°
3139,3140,WY,56039,Teton,Jackson,21294,10347.984,3995.379,572.266,220.953,10920.250,4216.332,+44.049321°,–110.588102°
3140,3141,WY,56041,Uinta,Evanston,21118,5390.450,2081.264,16.342,6.310,5406.791,2087.574,+41.284726°,–110.558947°
3141,3142,WY,56043,Washakie,Worland,8533,5797.815,2238.549,10.762,4.155,5808.577,2242.704,+43.878831°,–107.669052°


In [13]:
allCountyData_df = pd.DataFrame(countyTable[0])
allCountyData_df

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,1,AL,1001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,+32.536382°,–86.644490°
1,2,AL,1003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.190,437.527,5250.712,2027.311,+30.659218°,–87.746067°
2,3,AL,1005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,+31.870670°,–85.405456°
3,4,AL,1007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,+33.015893°,–87.127148°
4,5,AL,1009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,+33.977448°,–86.567246°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3139,WY,56037,Sweetwater,Green River,43806,27004.897,10426.649,166.887,64.436,27171.784,10491.085,+41.660339°,–108.875676°
3139,3140,WY,56039,Teton,Jackson,21294,10347.984,3995.379,572.266,220.953,10920.250,4216.332,+44.049321°,–110.588102°
3140,3141,WY,56041,Uinta,Evanston,21118,5390.450,2081.264,16.342,6.310,5406.791,2087.574,+41.284726°,–110.558947°
3141,3142,WY,56043,Washakie,Worland,8533,5797.815,2238.549,10.762,4.155,5808.577,2242.704,+43.878831°,–107.669052°


In [14]:
FIPS_to_ClimDiv_df = pd.read_csv('data/FIPS_to_ClimDiv.csv') 
FIPS_to_ClimDiv_df #this table came from an ncei table at the below url.
# https://www.ncei.noaa.gov/pub/data/cirs/climdiv/county-to-climdivs.txt

#I removed the middle column because the NCDC_FIPS_ID is no longer in use.
#the table is for the contiguous united state (CONUS), so we will have some guaranteed missing values--
#Hawaii does not have climate divisions assigned to it, and Alaska is normally reported separately from CONUS

,CLIMDIV_ID,FIPS
0,101,1033
1,101,1059
2,101,1077
3,101,1079
4,101,1083
...,...,...
3098,4808,56025
3099,4808,56031
3100,4809,56013
3101,4810,56001


In [15]:
FIPSandClimDiv_df = pd.merge(FIPS_to_ClimDiv_df, allCountyData_df, on='FIPS', how='outer')

In [16]:
FIPSandClimDiv_df

,CLIMDIV_ID,FIPS,Sort [1],State,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
0,101.0,1033,17.0,AL,Colbert,Tuscumbia,54428.0,1534.877,592.619,76.431,29.510,1611.308,622.129,+34.703112°,–87.801457°
1,101.0,1059,30.0,AL,Franklin,Russellville,31704.0,1641.588,633.821,32.898,12.702,1674.486,646.523,+34.441988°,–87.842815°
2,101.0,1077,39.0,AL,Lauderdale,Florence,92709.0,1729.328,667.697,138.027,53.293,1867.355,720.990,+34.904122°,–87.650997°
3,101.0,1079,40.0,AL,Lawrence,Moulton,34339.0,1788.847,690.678,68.688,26.521,1857.535,717.199,+34.529776°,–87.321865°
4,101.0,1083,42.0,AL,Limestone,Athens,82782.0,1450.228,559.936,122.382,47.252,1572.610,607.188,+34.810239°,–86.981400°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,NaN,26083,1273.0,MI,Keweenaw,Eagle River,2156.0,1398.883,540.112,14053.476,5426.078,15452.359,5966.190,+47.681981°,–88.148802°
3141,NaN,46102,2418.0,SD,Oglala Lakota [11],— [11],13586.0,5423.170,2093.898,7.126,2.751,5430.296,2096.649,+43.341937°,–102.559480°
3142,NaN,48007,2527.0,TX,Aransas,Rockport,23158.0,652.869,252.074,714.621,275.917,1367.490,527.991,+28.104225°,–96.977983°
3143,NaN,51735,2943.0,VA,Poquoson [9],—,12150.0,39.670,15.317,163.452,63.109,203.122,78.426,+37.128360°,–76.303534°


In [17]:
FIPSandClimDiv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3145 entries, 0 to 3144
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CLIMDIV_ID          3103 non-null   float64
 1   FIPS                3145 non-null   int64  
 2   Sort [1]            3143 non-null   float64
 3   State               3143 non-null   object 
 4   County [2]          3143 non-null   object 
 5   County Seat(s) [3]  3143 non-null   object 
 6   Population(2010)    3143 non-null   float64
 7   Land Areakm²        3143 non-null   float64
 8   Land Areami²        3143 non-null   float64
 9   Water Areakm²       3143 non-null   object 
 10  Water Areami²       3143 non-null   object 
 11  Total Areakm²       3143 non-null   float64
 12  Total Areami²       3143 non-null   float64
 13  Latitude            3143 non-null   object 
 14  Longitude           3143 non-null   object 
dtypes: float64(7), int64(1), object(7)
memory usage: 393.1+

In [18]:
FIPSandClimDiv_df.isna().sum()

CLIMDIV_ID            42
FIPS                   0
Sort [1]               2
State                  2
County [2]             2
County Seat(s) [3]     2
Population(2010)       2
Land Areakm²           2
Land Areami²           2
Water Areakm²          2
Water Areami²          2
Total Areakm²          2
Total Areami²          2
Latitude               2
Longitude              2
dtype: int64

The FIPS need to be set as 5 digit codes, so I will need to get a 0 in front of the states that have 4 digit FIPS codes.
Best bet is to format the fips column as strings and add leading zeros afterwards.
Should be the same solution for the climate divisions, but you might want to clean up the nulls for that column ahead of changing anything

In [19]:
FIPSandClimDiv_df['FIPS'] = FIPSandClimDiv_df['FIPS'].map(str)
FIPSandClimDiv_df.info() #great, FIPS is an object, now lets add some leading zeros

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3145 entries, 0 to 3144
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CLIMDIV_ID          3103 non-null   float64
 1   FIPS                3145 non-null   object 
 2   Sort [1]            3143 non-null   float64
 3   State               3143 non-null   object 
 4   County [2]          3143 non-null   object 
 5   County Seat(s) [3]  3143 non-null   object 
 6   Population(2010)    3143 non-null   float64
 7   Land Areakm²        3143 non-null   float64
 8   Land Areami²        3143 non-null   float64
 9   Water Areakm²       3143 non-null   object 
 10  Water Areami²       3143 non-null   object 
 11  Total Areakm²       3143 non-null   float64
 12  Total Areami²       3143 non-null   float64
 13  Latitude            3143 non-null   object 
 14  Longitude           3143 non-null   object 
dtypes: float64(7), object(8)
memory usage: 393.1+ KB


In [20]:
#finding the two rows with NaNs 
FIPSandClimDiv_df[FIPSandClimDiv_df['State'].isna()]

,CLIMDIV_ID,FIPS,Sort [1],State,County [2],County Seat(s) [3],Population(2010),Land Areakm²,Land Areami²,Water Areakm²,Water Areami²,Total Areakm²,Total Areami²,Latitude,Longitude
2349,3905.0,46113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2886,4405.0,51560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have two FIPS that are missing counties.
First is 46113 for Shannon county South Dakota. This was renamed Oglala Lakota County in 2015 and is under FIPS 46102 now.
The second is for Clifton Forge City county, Virginia. This had been an independent city (which is considered at the county-level in Virginia), but has since incorporated into a county.

We'll leave these two be for now. Shannon County may be present in older drought or weather data. Clifton Forge City changed after the 2000 census, so it may be long gone from any of the data we are looking at. We may need to merge Shannon county's dat with Oglala Lakota county(or overwrite the name and FIPS, at least.)

In [21]:
FIPSandClimDiv_df.to_csv('roughFIPSandClimDiv_df.csv', index=False)

In [22]:
FIPSandClimDiv_df['FIPS'] = FIPSandClimDiv_df['FIPS'].str.zfill(5) 
FIPSandClimDiv_df['FIPS']
#after digging through some of the pandas documentation on str, we have a winner

0       01033
1       01059
2       01077
3       01079
4       01083
        ...  
3140    26083
3141    46102
3142    48007
3143    51735
3144    53029
Name: FIPS, Length: 3145, dtype: object

Now we can loop through the FIPS column to get drought data for each county. Let's make a list from the FIPS, and a short, sub-list to test it out with the API.

In [23]:
FIPSlist = FIPSandClimDiv_df['FIPS'].tolist()
len(FIPSlist) #great, 3,145 counties

3145

In [24]:
shortFIPSlist = FIPSlist[:6]
shortFIPSlist

['01033', '01059', '01077', '01079', '01083', '01089']

# US Drought Monitor
---
The US Drought Monitor is hosted on the University of Nebraska, Lincoln website. It uses a variety of government data sources to determine the how many square miles in a county (or state, or several other areas) are experiencing drought. The level of drought is also available, ranging from abnormal dryness to exceptional drought. The Drought monitor has an API that can be accessed without an API key.


The API format is 'https://usdmdataservices.unl.edu/api/[area]/[statistics type]?aoi=[aoi]&startdate=[startdate]&enddate=[enddate]&statisticsType=[statistics type]'

[area]

ClimateDivisionStatistics for climate divisions

CountyStatistics for counties 

StateStatistics for states 

[statisticstype]

GetDroughtSeverityStatisticsByArea

[aoi]

Single state (by state fips code, so 27 for Minnesota)

or a single county (by county fips code, so 27053 for Hennepin county, MN)

or a single climate division (climate division list


[startdate] and [enddate]

m/d/yyyy format, 1/1/2012 and 12/31/2021 respectively

[statisticstype] (again)

The difference between statisticsType=1 and statistticsType=2 is type 1 is cumulative (so Drought level 0 would include all of the square milage at D1, D2, D3, and D4)
Type=2 would give the discrete (they called it categorical) number of sq miles at each level of drought severity.
**statistticsType=2 should be best for our needs**

Our county-level urls with 'countyfip' as the for loop object will be structured as:

'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByArea?aoi=[aoi]&startdate=[startdate]&enddate=[enddate]&statisticsType=[statistics type]'

In [25]:
testFIPS = list()

for fip in shortFIPSlist:

    # Make a request to the API
    url = f'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByArea?aoi={fip}&startdate=1/1/2012&enddate=12/31/2021&statisticsType=2'
    r = requests.get(url)
    time.sleep(3)

    # Add the result to a list of JSON
    testFIPS.append(r.json())
    
print(f'We have ten years of drought data for {len(testFIPS)} counties')

We have ten years of drought data for 6 counties


In [26]:
df_testFIPS = []
for i, j in enumerate(testFIPS, start=0):
    df=pd.DataFrame(j)
    df_testFIPS.append(df)
#I referenced the below link to figure out how to loop through my json results to set up dataframes:
# https://towardsdatascience.com/concatenate-multiple-and-messy-dataframes-efficiently-80847b4da12b
# and used the below code to concatenate these dataframes

In [27]:
df_testAllFIPS = pd.concat([df_testFIPS[i] for i in range(len(df_testFIPS))], ignore_index=True)
df_testAllFIPS #ok, that worked, here goes nothing

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20211228,01033,Colbert County,AL,371.40,253.63,0.00,0.00,0.00,0.00,2021-12-28,2022-01-03,2
1,20211221,01033,Colbert County,AL,592.73,32.30,0.00,0.00,0.00,0.00,2021-12-21,2021-12-27,2
2,20211214,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2
3,20211207,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-12-07,2021-12-13,2
4,20211130,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-11-30,2021-12-06,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,20120124,01089,Madison County,AL,812.19,0.00,0.00,0.00,0.00,0.00,2012-01-24,2012-01-30,2
3134,20120117,01089,Madison County,AL,812.19,0.00,0.00,0.00,0.00,0.00,2012-01-17,2012-01-23,2
3135,20120110,01089,Madison County,AL,812.19,0.00,0.00,0.00,0.00,0.00,2012-01-10,2012-01-16,2
3136,20120103,01089,Madison County,AL,812.19,0.00,0.00,0.00,0.00,0.00,2012-01-03,2012-01-09,2


The test drive worked well--we'll move on to pulling all of the counties in our list.

In [28]:
FIPS = list()

for fip in FIPSlist:
    print(f"Scraping data for fip number {fip}...", end="\r")
    # Make a request to the API
    url = f'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByArea?aoi={fip}&startdate=1/1/2012&enddate=12/31/2021&statisticsType=2'
    r = requests.get(url)
    time.sleep(1)

    # Add the result to a list of JSON
    FIPS.append(r.json())
    
print(f'We have ten years of drought data for {len(FIPS)} counties')

We have ten years of drought data for 3145 counties


In [29]:
df_FIPS = [] #one giant for loop for all of the drought dfs
for i, j in enumerate(FIPS, start=0):
    df=pd.DataFrame(j)
    df_FIPS.append(df)

In [30]:
df_droughtFIPS = pd.concat([df_FIPS[i] for i in range(len(df_FIPS))], ignore_index=True)
df_droughtFIPS 

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20211228,01033,Colbert County,AL,371.40,253.63,0.00,0.00,0.00,0.00,2021-12-28,2022-01-03,2
1,20211221,01033,Colbert County,AL,592.73,32.30,0.00,0.00,0.00,0.00,2021-12-21,2021-12-27,2
2,20211214,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-12-14,2021-12-20,2
3,20211207,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-12-07,2021-12-13,2
4,20211130,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2021-11-30,2021-12-06,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1642215,20120124,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2012-01-24,2012-01-30,2
1642216,20120117,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2012-01-17,2012-01-23,2
1642217,20120110,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2012-01-10,2012-01-16,2
1642218,20120103,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2012-01-03,2012-01-09,2


In [31]:
df_droughtFIPS.info() #wonderful! everything is set as an object which is ok for now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642220 entries, 0 to 1642219
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   MapDate            1642220 non-null  object
 1   FIPS               1642220 non-null  object
 2   County             1642220 non-null  object
 3   State              1642220 non-null  object
 4   None               1642220 non-null  object
 5   D0                 1642220 non-null  object
 6   D1                 1642220 non-null  object
 7   D2                 1642220 non-null  object
 8   D3                 1642220 non-null  object
 9   D4                 1642220 non-null  object
 10  ValidStart         1642220 non-null  object
 11  ValidEnd           1642220 non-null  object
 12  StatisticFormatID  1642220 non-null  object
dtypes: object(13)
memory usage: 162.9+ MB


In [32]:
df_droughtFIPS.to_csv('roughDroughtFIPS.csv', index=False) #spit this out to a csv

Because of the long wait involved in pulling data from the API, we can split the process into two pieces with 10 years of data for each piece. This will also make writing and reading the data out of/into dataframes more accessable--each piece contains ~1.6 million rows of data.

In [33]:
FIPS2 = list()

for fip in FIPSlist:
    print(f"Scraping data for fip number {fip}...", end="\r")
    # Make a request to the API
    url = f'https://usdmdataservices.unl.edu/api/CountyStatistics/GetDroughtSeverityStatisticsByArea?aoi={fip}&startdate=1/1/2002&enddate=12/31/2011&statisticsType=2'
    r = requests.get(url)
    time.sleep(1)

    # Add the result to a list of JSON
    FIPS2.append(r.json())
    
print(f'We have ten years of drought data for {len(FIPS2)} counties')

We have ten years of drought data for 3145 counties


In [34]:
df_FIPS2 = [] #one giant for loop for all of the drought dfs
for i, j in enumerate(FIPS2, start=0):
    df=pd.DataFrame(j)
    df_FIPS2.append(df)

In [35]:
df_droughtFIPS2 = pd.concat([df_FIPS2[i] for i in range(len(df_FIPS2))], ignore_index=True)
df_droughtFIPS2 

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,ValidStart,ValidEnd,StatisticFormatID
0,20111227,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2011-12-27,2012-01-02,2
1,20111220,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2011-12-20,2011-12-26,2
2,20111213,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2011-12-13,2011-12-19,2
3,20111206,01033,Colbert County,AL,625.03,0.00,0.00,0.00,0.00,0.00,2011-12-06,2011-12-12,2
4,20111129,01033,Colbert County,AL,159.05,465.98,0.00,0.00,0.00,0.00,2011-11-29,2011-12-05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639075,20020129,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2002-01-29,2002-02-04,2
1639076,20020122,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2002-01-22,2002-01-28,2
1639077,20020115,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2002-01-15,2002-01-21,2
1639078,20020108,53029,Island County,WA,213.97,0.00,0.00,0.00,0.00,0.00,2002-01-08,2002-01-14,2


In [36]:
df_droughtFIPS2.info() #wonderful! everything is set as an object which is ok for now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639080 entries, 0 to 1639079
Data columns (total 13 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   MapDate            1639080 non-null  object
 1   FIPS               1639080 non-null  object
 2   County             1639080 non-null  object
 3   State              1639080 non-null  object
 4   None               1639080 non-null  object
 5   D0                 1639080 non-null  object
 6   D1                 1639080 non-null  object
 7   D2                 1639080 non-null  object
 8   D3                 1639080 non-null  object
 9   D4                 1639080 non-null  object
 10  ValidStart         1639080 non-null  object
 11  ValidEnd           1639080 non-null  object
 12  StatisticFormatID  1639080 non-null  object
dtypes: object(13)
memory usage: 162.6+ MB


In [37]:
df_droughtFIPS2.to_csv('roughDroughtFIPS2.csv', index=False) #spit this out to a csv

# USDA Census of Agriculture Data
---
The USDA has roughly 170 years of data hosted on it's quick stats application. This includes data from the Census of Agriculture that is performed every 5 years and data from the Survey of Agriculture, which is performed every year. The Census contains a great deal of information--the full Census as a .txt file is over a gigabyte. 

The quick stats application is the most straightforward way to pull discrete parts from the full census for multiple years and multiple states/counties. Because the application allows the user to download csv files based on the parameters the user sets.

This was the method used to generate the csv file used in the fourth notebook (Soil, Data Wrangling part 3: USDA Census of Agriculture): COA_ChemFertOps&Acres_Counties.csv. 

The same data can be generated by setting the following parameters in the quick stats application:

https://quickstats.nass.usda.gov/

Program:
* Select CENSUS

Sector:
* Skip, no selection required

Group:
* Skip, no selection required

Commodity:
* Using the ctrl-left click or cmd-left click to select multiple parameters, select:

CHEMICAL TOTALS, FARM OPERATIONS; FERTILIZER TOTALS, OPERATORS

Category:
* Skip, no selection required

Data Item:

* Using the ctrl-left click or cmd-left click to select multiple parameters, select:

CHEMICAL TOTALS - EXPENSE, MEASURED IN DOLLARS; CHEMICAL TOTALS - OPERATIONS WITH EXPENSE; FARM OPERATIONS - ACRES OPERATED; FARM OPERATIONS - NUMBER OF OPERATIONS; FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - EXPENSE, MEASURED IN DOLLARS; FERTILIZER TOTALS, INCL LIME & SOIL CONDITIONERS - OPERATIONS WITH EXPENSE;

Domain:
* select TOTAL;

Geographic Level:
* select COUNTY

State:
* Using the ctrl-left click or cmd-left click to select multiple parameters, select:
ARKANSAS; CALIFORNIA; FLORIDA; GEORGIA; ILLINOIS; INDIANA; IOWA; KANSAS; MINNESOTA; MISSOURI; NEBRASKA; NORTH CAROLINA; TEXAS; WASHINGTON; WISCONSIN;

Ag District:
* Skip, no selection required

County:
* Skip, no selection required

Year:
* select 2017, 2012, 2007, 2002

Period Type:
* Skip, no selection required

There will be 36,350 records available. Click Get Data to advance to a readout of the selected data. Click spreadsheet to download a csv file of the filtered data.